In [16]:
import sqlite3  # SQLite lets you treat a CSV file like a database table.
import pandas as pd

# Load CSV with pandas
daily_activity = pd.read_csv("dailyActivity_merged.csv")
sleep_day = pd.read_csv("sleepDay_merged.csv")
hourly_cal = pd.read_csv("hourlyCalories_merged.csv")
heart_rate = pd.read_csv("heartrate_seconds_merged.csv")

# Connect to SQLite (creates a db file in memory)
conn = sqlite3.connect(":memory:")

# Write DataFrame to SQL table
daily_activity.to_sql("daily_activity", conn, index=False, if_exists="replace")
sleep_day.to_sql("sleep_day", conn, index=False, if_exists="replace")
hourly_cal.to_sql("hourly_cal", conn, index=False, if_exists="replace")
heart_rate.to_sql("heart_rate", conn, index=False, if_exists="replace")




2483658

In [2]:
# See the table structure
query = """
PRAGMA table_info(daily_activity);
"""
result = pd.read_sql_query(query, conn)
result

,cid,name,type,notnull,dflt_value,pk
0,0,Id,INTEGER,0,None,0
1,1,ActivityDate,TEXT,0,None,0
2,2,TotalSteps,INTEGER,0,None,0
3,3,TotalDistance,REAL,0,None,0
4,4,TrackerDistance,REAL,0,None,0
5,5,LoggedActivitiesDistance,REAL,0,None,0
6,6,VeryActiveDistance,REAL,0,None,0
7,7,ModeratelyActiveDistance,REAL,0,None,0
8,8,LightActiveDistance,REAL,0,None,0
9,9,SedentaryActiveDistance,REAL,0,None,0


In [3]:
# See the table structure
query = """
PRAGMA table_info(sleep_day);
"""
result = pd.read_sql_query(query, conn)
result

,cid,name,type,notnull,dflt_value,pk
0,0,Id,INTEGER,0,None,0
1,1,SleepDay,TEXT,0,None,0
2,2,TotalSleepRecords,INTEGER,0,None,0
3,3,TotalMinutesAsleep,INTEGER,0,None,0
4,4,TotalTimeInBed,INTEGER,0,None,0


In [4]:
# See the table structure
query = """
PRAGMA table_info(hourly_cal);
"""
result = pd.read_sql_query(query, conn)
result

,cid,name,type,notnull,dflt_value,pk
0,0,Id,INTEGER,0,None,0
1,1,ActivityHour,TEXT,0,None,0
2,2,Calories,INTEGER,0,None,0


In [17]:
# See the table structure
query = """
PRAGMA table_info(heart_rate);
"""
result = pd.read_sql_query(query, conn)
result

,cid,name,type,notnull,dflt_value,pk
0,0,Id,INTEGER,0,None,0
1,1,Time,TEXT,0,None,0
2,2,Value,INTEGER,0,None,0


In [5]:
# check date format
print(daily_activity['ActivityDate'].head())

0    4/12/2016
1    4/13/2016
2    4/14/2016
3    4/15/2016
4    4/16/2016
Name: ActivityDate, dtype: object


In [6]:
print(sleep_day["SleepDay"].head())

0    4/12/2016 12:00:00 AM
1    4/13/2016 12:00:00 AM
2    4/15/2016 12:00:00 AM
3    4/16/2016 12:00:00 AM
4    4/17/2016 12:00:00 AM
Name: SleepDay, dtype: object


In [7]:
print(hourly_cal['ActivityHour'].head())

0    4/12/2016 12:00:00 AM
1     4/12/2016 1:00:00 AM
2     4/12/2016 2:00:00 AM
3     4/12/2016 3:00:00 AM
4     4/12/2016 4:00:00 AM
Name: ActivityHour, dtype: object


In [8]:
# convert the date format to YYYY-MM-DD to be able to extract date parts from the column using sqlite
daily_activity['ActivityDate'] = pd.to_datetime(daily_activity['ActivityDate']).dt.strftime('%Y-%m-%d')
# Write DataFrame to SQL table
daily_activity.to_sql("daily_activity", conn, index=False, if_exists="replace")
print(daily_activity['ActivityDate'].head())

0    2016-04-12
1    2016-04-13
2    2016-04-14
3    2016-04-15
4    2016-04-16
Name: ActivityDate, dtype: object


In [9]:
sleep_day["SleepDay"] = pd.to_datetime(sleep_day["SleepDay"]).dt.strftime('%Y-%m-%d')
sleep_day.to_sql("sleep_day", conn, index=False, if_exists="replace")
print(sleep_day["SleepDay"].head())

0    2016-04-12
1    2016-04-13
2    2016-04-15
3    2016-04-16
4    2016-04-17
Name: SleepDay, dtype: object


C:\Users\LAP TECH\AppData\Local\Temp\ipykernel_19240\2519704248.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sleep_day["SleepDay"] = pd.to_datetime(sleep_day["SleepDay"]).dt.strftime('%Y-%m-%d')


In [10]:
hourly_cal["ActivityHour"] = pd.to_datetime(hourly_cal["ActivityHour"], format="%m/%d/%Y %I:%M:%S %p")
hourly_cal["ActivityHour"] = hourly_cal["ActivityHour"].dt.strftime('%Y-%m-%d %H:%M:%S')
hourly_cal.to_sql("hourly_cal", conn, index=False, if_exists="replace")
print(hourly_cal["ActivityHour"].head())

0    2016-04-12 00:00:00
1    2016-04-12 01:00:00
2    2016-04-12 02:00:00
3    2016-04-12 03:00:00
4    2016-04-12 04:00:00
Name: ActivityHour, dtype: object


# First insight
See the monthly usage of the FitBit tracker, how many users used the devices every month

In [11]:
query = """
SELECT STRFTIME('%m', ActivityDate) as month, COUNT(Id) as num_users
FROM daily_activity
GROUP BY STRFTIME('%m', ActivityDate);
"""
result = pd.read_sql_query(query, conn)
result

,month,num_users
0,04,611
1,05,329


# Second insight
Categorize users into very active, moderately active and lightly active users based on their activity distance.

In [12]:
query = """
SELECT Id, 
    sum(TotalDistance) as total_distance_moved,
    CASE
        WHEN sum(VeryActiveDistance) > sum(ModeratelyActiveDistance) AND sum(VeryActiveDistance) > sum(LightActiveDistance) THEN "Very active user"
        WHEN sum(LightActiveDistance) > sum(ModeratelyActiveDistance) AND sum(VeryActiveDistance) < sum(LightActiveDistance) THEN "Lightly active user"
        ELSE "Moderately active user"
    END as User_type
FROM daily_activity
GROUP BY Id;
"""
result = pd.read_sql_query(query, conn)
result


,Id,total_distance_moved,User_type
0,1503960366,242.099999,Lightly active user
1,1624580081,121.360001,Lightly active user
2,1644430081,158.860001,Lightly active user
3,1844505072,52.890000,Lightly active user
4,1927972279,19.670000,Lightly active user
5,2022484408,250.609998,Lightly active user
6,2026352035,107.100000,Lightly active user
7,2320127002,98.819999,Lightly active user
8,2347167796,114.400000,Lightly active user
9,2873212765,158.149999,Lightly active user


# Third insight
See the day that has the longest sleep durantion and time in bed, the day most of the people rest in it.

In [13]:
query = """
SELECT CASE STRFTIME('%w', SleepDay)
            WHEN '0' THEN 'Sunday'
            WHEN '1' THEN 'Monday'
            WHEN '2' THEN 'Tuesday'
            WHEN '3' THEN 'Wednesday'
            WHEN '4' THEN 'Thursday'
            WHEN '5' THEN 'Friday'
            WHEN '6' THEN 'Saturday'
       END AS weekday_name
    , avg(TotalTimeInBed) as avg_time_in_bed
FROM sleep_day
GROUP BY strftime('%w', SleepDay)
ORDER BY avg_time_in_bed DESC
LIMIT 1;
"""
result = pd.read_sql_query(query, conn)
result

,weekday_name,avg_time_in_bed
0,Sunday,503.509091


# Fourth insight
See the hour that every user burns the most calories in 

In [14]:
query = """
with cte as(
    SELECT Id, STRFTIME('%H', ActivityHour) as hour, SUM(Calories) as num_calories
    FROM hourly_cal
    GROUP BY Id, STRFTIME('%H', ActivityHour)
    )
SELECT * 
FROM cte as c
WHERE num_calories = (SELECT MAX(num_calories)
                    FROM cte as c2
                    WHERE c2.Id = c.Id);
"""
result = pd.read_sql_query(query, conn)
result

,Id,hour,num_calories
0,1503960366,18,3854
1,1624580081,19,2273
2,1644430081,19,5668
3,1844505072,11,2428
4,1927972279,11,3206
5,2022484408,09,6846
6,2026352035,15,2533
7,2320127002,20,2509
8,2347167796,18,2537
9,2873212765,08,4364


# Fifth insight
Find the user with the highest average heart rate

In [23]:
query = """
SELECT Id, avg(Value) as avg_heart_rate_value
FROM heart_rate
GROUP BY Id
ORDER BY avg_heart_rate_value desc
LIMIT 1;
"""
result = pd.read_sql_query(query, conn)
result

,Id,avg_heart_rate_value
0,2026352035,93.776305
